# Projet maison n° 5

On s'intéresse au fichier sur la Couverture 2G, 3G, 4G en France par opérateur de juillet 2015 fourni par l'ARCEP : "couverture-2g-3g-4g-en-france-par-operateur-juillet-2015.csv".

[°] Ce fichier donne le taux de couverture par commune, par opérateur (Orange France, Bouygues Telecom, SFR, Free Mobile, Tout Opérateur), par type de couverture (Couverture population, Couverture surfacique) et par réseau (2G, 3G, 4G).

1. Chargez le fichier dans un DataFrame.
2. Transformez ce DataFrame large en un DataFrame long en isolant les colonnes d'identification d'une part et les colonnes de valeurs de couverture d'autre part.
3. Rajoutez 3 colonnes "Opérateur", "Type Couverture" et "Réseau" calculées à partir de la colonne "variable" obtenue en 2 (utilisez les valeurs indiquées entre parenthèses ci-dessus [°], ces valeurs sont utilisées dans les tests).
4. A partir de ce DataFrame, obtenez un objet de type Series dont les valeurs correspondent à la colonne "value" obtenue en 2 et dont le multi-index contient les autres colonnes du DataFrame sauf la colonne "variable" obtenue en 2. Puis remontez les index "Opérateur", "Type Couverture" et "Réseau" en colonnes multi-indexées. Vous devez obtenir un DataFrame avec 7 niveaux d'index et 3 niveaux de colonnes.
5. Chargez le DataFrame des communes déjà utilisé ("correspondance-code-insee-code-postal.csv") et effectuez une jointure sur le code INSEE avec le DataFrame obtenu en 3.
6. Pour le réseau 4G et la couverture surfacique, calculez la moyenne des couvertures par statut de commune et par opérateur. Pour quel couple (statut, opérateur) a-t-on la moyenne de la couverture la plus faible ?

**Nota bene** : les différentes instructions doivent être réalisées dans les fonctions prédéfinies pour passer les tests. Il est néanmoins possible de définir des fonctions annexes appelées par les fonctions prédéfinies.

In [48]:
# import
import pandas as pd
import numpy as np
import re

path = "couverture-2g-3g-4g-en-france-par-operateur-juillet-2015.csv"

In [36]:
# 1) Chargez le fichier dans un DataFrame
def load_data():
    df = pd.read_csv(path, delimiter=';')
    return df

In [37]:
# load_data
df = load_data()

In [38]:
df.head()

,CODE POSTAL,CODE INSEE,NOM COMMUNE,CODE DEPARTEMENT,SURFACE COMMUNE (km²),POPULATION COMMUNE,Orange France Couverture population 4G,Bouygues Telecom Couverture population 4G,SFR Couverture population 4G,Free Mobile Couverture population 4G,...,Bouygues Telecom Couverture population 2G,SFR Couverture population 2G,Free Mobile Couverture population 2G,Par au moins un opérateur Couverture population 2G,Orange France Couverture surfacique 2G,Bouygues Telecom Couverture surfacique 2G,SFR Couverture surfacique 2G,Free Mobile Couverture surfacique 2G,Par au moins un opérateur Couverture surfacique 2G,coordonnees
0,17510,17105,CHIVES,17,20.642005,355.32,0.00,0.0,0.0,0.0,...,1.00,1.0,1.0,1.0,1.00,0.99,1.0,1.00,1.0,"45.9508835378, -0.121918052323"
1,17600,17112,LA CLISSE,17,4.896771,591.78,0.09,0.0,0.0,0.0,...,1.00,1.0,1.0,1.0,1.00,1.00,1.0,1.00,1.0,"45.7320429784, -0.75493581331"
2,17150,17116,CONSAC,17,8.915911,239.74,0.00,0.0,0.0,0.0,...,1.00,1.0,1.0,1.0,1.00,1.00,1.0,1.00,1.0,"45.4182495786, -0.588917528415"
3,17130,17118,CORIGNAC,17,10.850210,358.69,0.00,0.0,0.0,0.0,...,0.63,1.0,1.0,1.0,0.99,0.85,1.0,0.99,1.0,"45.2448604983, -0.39280322021"
4,17330,17124,COURANT,17,15.586269,371.00,0.00,0.0,0.0,0.0,...,1.00,1.0,1.0,1.0,1.00,1.00,1.0,1.00,1.0,"46.0375312829, -0.587834905531"


In [41]:
# 2) Transformez ce DataFrame large en un DataFrame long
def data_wide2long():
    df = load_data()
    columns_values = df.columns[6:-1]
    columns_id = list(df.columns[:6]) + list([df.columns[-1]])
    df = df.melt(id_vars=columns_id, value_vars=columns_values)
    return df

In [63]:
# data_wide2long
data_wide2long().head()

,CODE POSTAL,CODE INSEE,NOM COMMUNE,CODE DEPARTEMENT,SURFACE COMMUNE (km²),POPULATION COMMUNE,coordonnees,variable,value
0,17510,17105,CHIVES,17,20.642005,355.32,"45.9508835378, -0.121918052323",Orange France Couverture population 4G,0.00
1,17600,17112,LA CLISSE,17,4.896771,591.78,"45.7320429784, -0.75493581331",Orange France Couverture population 4G,0.09
2,17150,17116,CONSAC,17,8.915911,239.74,"45.4182495786, -0.588917528415",Orange France Couverture population 4G,0.00
3,17130,17118,CORIGNAC,17,10.850210,358.69,"45.2448604983, -0.39280322021",Orange France Couverture population 4G,0.00
4,17330,17124,COURANT,17,15.586269,371.00,"46.0375312829, -0.587834905531",Orange France Couverture population 4G,0.00


In [143]:
# 3) Rajoutez 3 colonnes "Opérateur", "Type Couverture" et "Réseau"
def data_add3columns():
    df = data_wide2long()
    df['Opérateur'] = df.variable.apply(lambda x: re.search('(.+)\sCouverture', x).group(1))
    df["Type Couverture"] = df.variable.apply(lambda x: re.search('(Couverture.+)\s\d', x).group(1))
    df['Réseau'] = df.variable.apply(lambda x: re.search('(\dG)', x).group(1))
    df['Opérateur'] = df['Opérateur'].apply(lambda x: "Tout Opérateur" if 'Par' in x else x)
    return df

In [144]:
# data_add3columns
data_add3columns().head()

,CODE POSTAL,CODE INSEE,NOM COMMUNE,CODE DEPARTEMENT,SURFACE COMMUNE (km²),POPULATION COMMUNE,coordonnees,variable,value,Opérateur,Type Couverture,Réseau
0,17510,17105,CHIVES,17,20.642005,355.32,"45.9508835378, -0.121918052323",Orange France Couverture population 4G,0.00,Orange France,Couverture population,4G
1,17600,17112,LA CLISSE,17,4.896771,591.78,"45.7320429784, -0.75493581331",Orange France Couverture population 4G,0.09,Orange France,Couverture population,4G
2,17150,17116,CONSAC,17,8.915911,239.74,"45.4182495786, -0.588917528415",Orange France Couverture population 4G,0.00,Orange France,Couverture population,4G
3,17130,17118,CORIGNAC,17,10.850210,358.69,"45.2448604983, -0.39280322021",Orange France Couverture population 4G,0.00,Orange France,Couverture population,4G
4,17330,17124,COURANT,17,15.586269,371.00,"46.0375312829, -0.587834905531",Orange France Couverture population 4G,0.00,Orange France,Couverture population,4G


In [150]:
# 4) Transformez ce DataFrame en multi-index
def data_multi_index():
    df = data_add3columns()
    df = df.drop(columns=['variable'])
    columns = list(df.columns)
    columns.remove('value')
    df = df.set_index(keys=columns)
    df = df.unstack(level=['Opérateur', 'Type Couverture', 'Réseau'])
    df = df.droplevel(level=0, axis=1)
    return df

In [151]:
# data_multi_index
data_multi_index().head()

Opérateur                                                                                                                                   Orange France  \
Type Couverture                                                                                                                     Couverture population   
Réseau                                                                                                                                                 4G   
CODE POSTAL CODE INSEE NOM COMMUNE           CODE DEPARTEMENT SURFACE COMMUNE (km²) POPULATION COMMUNE coordonnees                                          
01000       01053      BOURG-EN-BRESSE       01               23.876482             38923.25           46.2051520382, 5.24602125501                  0.99   
            01344      SAINT-DENIS-LES-BOURG 01               13.006094             6163.17            46.2111998453, 5.18462806771                  1.00   
01090       01165      FRANCHELEINS          01               13.779201             1349.42            46.0735946256, 4.81293575559                  0.93   
            01169      GENOUILLEUX           01               3.759855              608.97             46.1201611728, 4.79288195016                  1.00   
            01183      GUEREINS              01               4.579867              1308.07            46.106624674, 4.78159420118                   1.00   

Opérateur                                                                                                                                Bouygues Telecom  \
Type Couverture                                                                                                                     Couverture population   
Réseau                                                                                                                                                 4G   
CODE POSTAL CODE INSEE NOM COMMUNE           CODE DEPARTEMENT SURFACE COMMUNE (km²) POPULATION COMMUNE coordonnees                                          
01000       01053      BOURG-EN-BRESSE       01               23.876482             38923.25           46.2051520382, 5.24602125501                  0.99   
            01344      SAINT-DENIS-LES-BOURG 01               13.006094             6163.17            46.2111998453, 5.18462806771                  1.00   
01090       01165      FRANCHELEINS          01               13.779201             1349.42            46.0735946256, 4.81293575559                  0.98   
            01169      GENOUILLEUX           01               3.759855              608.97             46.1201611728, 4.79288195016                  1.00   
            01183      GUEREINS              01               4.579867              1308.07            46.106624674, 4.78159420118                   1.00   

Opérateur                                                                                                                                             SFR  \
Type Couverture                                                                                                                     Couverture population   
Réseau                                                                                                                                                 4G   
CODE POSTAL CODE INSEE NOM COMMUNE           CODE DEPARTEMENT SURFACE COMMUNE (km²) POPULATION COMMUNE coordonnees                                          
01000       01053      BOURG-EN-BRESSE       01               23.876482             38923.25           46.2051520382, 5.24602125501                  0.97   
            01344      SAINT-DENIS-LES-BOURG 01               13.006094             6163.17            46.2111998453, 5.18462806771                  0.98   
01090       01165      FRANCHELEINS          01               13.779201             1349.42            46.0735946256, 4.81293575559                  0.76   
            01169      GENOUILLEUX           01               3.759855   

In [117]:
# 5) Effectuez une jointure sur le code INSEE
def data_join_geo():
    geo = pd.read_csv("correspondance-code-insee-code-postal.csv", delimiter=';')
    df = data_add3columns()
    df = pd.merge(df, geo, how='left', right_on='Code INSEE', left_on='CODE INSEE')
    df = df.drop(columns='Code INSEE')
    return df

In [119]:
# data_join_geo
data_join_geo().head()

,CODE POSTAL,CODE INSEE,NOM COMMUNE,CODE DEPARTEMENT,SURFACE COMMUNE (km²),POPULATION COMMUNE,coordonnees,variable,value,Opérateur,...,Superficie,Population,geo_point_2d,geo_shape,ID Geogla,Code Commune,Code Canton,Code Arrondissement,Code Département,Code Région
0,17510,17105,CHIVES,17,20.642005,355.32,"45.9508835378, -0.121918052323",Orange France Couverture population 4G,0.00,Orange France,...,2062.0,0.4,"45.9508835378, -0.121918052323","{""type"": ""Polygon"", ""coordinates"": [[[-0.10484...",22032,105,4,5,17,54
1,17600,17112,LA CLISSE,17,4.896771,591.78,"45.7320429784, -0.75493581331",Orange France Couverture population 4G,0.09,Orange France,...,524.0,0.6,"45.7320429784, -0.75493581331","{""type"": ""Polygon"", ""coordinates"": [[[-0.74528...",26593,112,36,4,17,54
2,17150,17116,CONSAC,17,8.915911,239.74,"45.4182495786, -0.588917528415",Orange France Couverture population 4G,0.00,Orange France,...,905.0,0.2,"45.4182495786, -0.588917528415","{""type"": ""Polygon"", ""coordinates"": [[[-0.60200...",34862,116,16,1,17,54
3,17130,17118,CORIGNAC,17,10.850210,358.69,"45.2448604983, -0.39280322021",Orange France Couverture population 4G,0.00,Orange France,...,1112.0,0.3,"45.2448604983, -0.39280322021","{""type"": ""Polygon"", ""coordinates"": [[[-0.41318...",27409,118,17,1,17,54
4,17330,17124,COURANT,17,15.586269,371.00,"46.0375312829, -0.587834905531",Orange France Couverture population 4G,0.00,Orange France,...,1574.0,0.4,"46.0375312829, -0.587834905531","{""type"": ""Polygon"", ""coordinates"": [[[-0.58528...",20543,124,12,5,17,54


In [136]:
# 6) Pour quels statut et opérateur a-t-on la couverture la plus faible ?
def data_get_worst_statut_operateur():
    df = data_join_geo()
    df = df[(df['Réseau'] == '4G') & (df['Type Couverture'] == 'Couverture surfacique')]
    worst_statut_operateur = df.groupby(['Statut', 'Opérateur']).mean().value.idxmin()
    return worst_statut_operateur

In [137]:
# data_get_worst_statut_operateur
data_get_worst_statut_operateur()

('Commune simple', 'Free Mobile')

In [138]:
import unittest

class Lesson5Tests(unittest.TestCase):
    
    def test_01_load_data(self):
        df = load_data()
        # test shape
        self.assertEqual(df.shape, (36594, 37))
        
    def test_02_data_wide2long(self):
        df = data_wide2long()
        # test shape
        self.assertEqual(df.shape, (1097820, 9))
        
    def test_03_data_add3columns(self):
        df = data_add3columns()
        # test "Opérateur" values
        s1 = set(df['Opérateur'].unique())
        s2 = set(['Orange France', 'Bouygues Telecom', 'SFR', 'Free Mobile', 'Tout Opérateur'])
        self.assertSetEqual(s1, s2)
        # test "Type Couverture" values
        s1 = set(df['Type Couverture'].unique())
        s2 = set(['Couverture population', 'Couverture surfacique'])
        self.assertSetEqual(s1, s2)
        # test "Réseau" values
        s1 = set(df['Réseau'].unique())
        s2 = set(['2G', '3G', '4G'])
        self.assertSetEqual(s1, s2)
        
    def test_04_data_multi_index(self):
        df = data_multi_index()
        # test shape
        self.assertEqual(df.shape, (36594, 30))
        # test multi-index de l'index
        self.assertEqual(len(df.index.levels), 7)
        # test multi-index des columns
        self.assertEqual(len(df.columns.levels), 3)
        
    def test_05_data_join_geo(self):
        df = data_join_geo()
        # test length
        self.assertEqual(len(df), 1097820)
        
    def test_06_data_get_worst_statut_operateur(self):
        statut_operateur = data_get_worst_statut_operateur()
        # test length
        self.assertEqual(statut_operateur, ('Commune simple', 'Free Mobile'))


In [152]:
# run tests
test_suite = unittest.makeSuite(Lesson5Tests)
runner = unittest.TextTestRunner(verbosity=2)
runner.run(test_suite)

test_01_load_data (__main__.Lesson5Tests) ... ok
test_02_data_wide2long (__main__.Lesson5Tests) ... ok
test_03_data_add3columns (__main__.Lesson5Tests) ... ok
test_04_data_multi_index (__main__.Lesson5Tests) ... ok
test_05_data_join_geo (__main__.Lesson5Tests) ... ok
test_06_data_get_worst_statut_operateur (__main__.Lesson5Tests) ... ok

----------------------------------------------------------------------
Ran 6 tests in 14.175s

OK


<unittest.runner.TextTestResult run=6 errors=0 failures=0>